In [1]:
import astropy.constants as c
import glob
from hciplot import plot_frames, plot_cubes
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

#from special.model_resampling import resample_model
#from special.utils_spec import find_nearest

from vip_hci.config import VLT_SPHERE_IFS
from vip_hci.fits import open_fits, write_fits, info_fits
from vip_hci.fm import (cube_inject_companions, firstguess, mcmc_negfc_sampling, normalize_psf,
                        show_walk_plot, show_corner_plot, confidence)
from vip_hci.metrics import snr, significance
from vip_hci.psfsub import median_sub, pca, pca_annular, pca_annulus, pca_grid
from vip_hci.preproc import find_scal_vector, frame_rescaling
from vip_hci.var import frame_center, dist, mask_circle

## Loading and visualizing the data

In [ ]:
cubename = '/home/cvargas/sphere/IFS/SPHERE_DC_DATA_AS209_IFS/HIP-82323_OBS_H_2021-04-06_ifs_convert_dc2_PUBLIC_305341/ifs_convert_dc2-IFS_SCIENCE_REDUCED_SPECTRAL_MASTER_CUBE-center_im.fits'
psfname = '/home/cvargas/sphere/IFS/SPHERE_DC_DATA_AS209_IFS/HIP-82323_OBS_H_2021-04-06_ifs_convert_dc2_PUBLIC_305341/ifs_convert_dc2-IFS_SCIENCE_PSF_MASTER_CUBE-median_unsat.fits'
lbdaname = '/home/cvargas/sphere/IFS/SPHERE_DC_DATA_AS209_IFS/HIP-82323_OBS_H_2021-04-06_ifs_convert_dc2_PUBLIC_305341/ifs_convert_dc2-IFS_SCIENCE_LAMBDA_INFO-lam.fits'
anglename = '/home/cvargas/sphere/IFS/SPHERE_DC_DATA_AS209_IFS/HIP-82323_OBS_H_2021-04-06_ifs_convert_dc2_PUBLIC_305341/ifs_convert_dc2-IFS_SCIENCE_PARA_ROTATION_CUBE-rotnth.fits'

cube = open_fits(cubename)
psf = open_fits(psfname)
lbda = open_fits(lbdaname)
derot_angles = -1*open_fits(anglename) # note: SPHERE DRH returns parallactic angles, while VIP routines take derotation angles as input

nch, nz, ny, nx = cube.shape